In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import pywt
import timeit
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from scipy import signal
from keras.layers import Dense, Flatten, Activation, TimeDistributed
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import AveragePooling1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.utils import np_utils
from keras.optimizers import SGD, Adam, Nadam
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
import keras.callbacks
from keras import initializers, regularizers
from keras.layers.pooling import GlobalAvgPool2D
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU, PReLU

Using TensorFlow backend.


In [2]:
np.random.seed(42)
dataset = sio.loadmat('dataset_1.mat')
heart_rate = dataset['heart_rate']
heart_rate = (heart_rate - 50) / 150
ss = np.zeros((3203, 1000, 1))
signal = dataset['signal']
ss[:, :, 0] = signal
print(ss.shape)
print(heart_rate.shape)

(3203, 1000, 1)
(3203, 1)


In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(ss, heart_rate, test_size=0.2, random_state=123)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(2562, 1000, 1)
(641, 1000, 1)
(2562, 1)
(641, 1)


In [4]:
# network design
model = Sequential()
# first part
model.add(Conv1D(64, 16, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# second part
model.add(Conv1D(64, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 16, padding='same'))
model.add(MaxPooling1D())
# third layers
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(192, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(192, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(192, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(192, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(256, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(256, 16, padding='same'))
model.add(MaxPooling1D())

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(256, 16, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(256, 16, padding='same'))
model.add(MaxPooling1D())
##
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Flatten())
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

#model.summary()

In [5]:
# optimization setup
optm = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=3*10**(-4))
sgd = SGD(lr=0.0005, momentum=0.9, decay=0.0, nesterov=False)
model.compile(
    loss='mae',
    optimizer=optm,
    metrics=['mse'])

In [6]:
filepath="weights.best1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', mode='auto', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

In [7]:
# training
model.fit(
    X_train, 
    Y_train,
    batch_size=128,
    epochs=3000,
    validation_split=0.2,
    shuffle=True,
    callbacks=callbacks_list,
    verbose=2)

Train on 2049 samples, validate on 513 samples
Epoch 1/3000


ResourceExhaustedError: OOM when allocating tensor with shape[16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/mul_283 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_2/read, training/Adam/Variable_132/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training/Adam/gradients/conv1d_4/convolution/Conv2D_grad/ShapeN/_895 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1125_training/Adam/gradients/conv1d_4/convolution/Conv2D_grad/ShapeN", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocati

In [ ]:
Y_pred = model.predict(X_test)
Y_test = Y_test * 150 + 50
Y_pred = Y_pred * 150 + 50
plt.figure
plt.scatter(Y_test, Y_pred)
Y_error = abs(Y_pred - Y_test)
print('The avarage predict error: ', np.mean(Y_error))
print('The mean squared error:', metrics.mean_squared_error(Y_test, Y_pred))

In [ ]:
Y_pred = model.predict(X_train)
Y_train = Y_train * 150 + 50
Y_pred = Y_pred * 150 + 50
plt.figure
plt.scatter(Y_train, Y_pred)
Y_error = abs(Y_pred - Y_train)
print('The avarage predict error: ', np.mean(Y_error))
print('The mean squared error:', metrics.mean_squared_error(Y_train, Y_pred))

In [ ]:
# Using the best model
model.load_weights("weights.best1.hdf5")

In [ ]:
Y_pred = model.predict(X_test)
Y_pred = Y_pred * 150 + 50
plt.figure
plt.scatter(Y_test, Y_pred)
Y_error = abs(Y_pred - Y_test)
print('The avarage predict error: ', np.mean(Y_error))
print('The mean squared error:', metrics.mean_squared_error(Y_test, Y_pred))

In [ ]:
Y_pred = model.predict(X_train)
#Y_train = Y_train * 150 + 50
Y_pred = Y_pred * 150 + 50
plt.figure
plt.scatter(Y_train, Y_pred)
Y_error = abs(Y_pred - Y_train)
print('The avarage predict error: ', np.mean(Y_error))
print('The mean squared error:', metrics.mean_squared_error(Y_train, Y_pred))